In [6]:
import getpass
import os
import pprint
import pandas as pd
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
#os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Define evaluation function
def evaluate_criteria(parser, testcases_df, eval_llm, tested_llm, prompt):
    eval_store_data = []
    #Iterate over test cases and call LLM(s) for answer
    
    for index, row in testcases_df.iterrows():
        question = row["question"]
        reference = row["reference"]

        # # Get answer from LLM
        # answer = tested_llm.invoke([HumanMessage(content=question)]).content
        
        # Get answer from reference
        answer = row["nps_advisor_answer"]
        
        # Evaluate response
        eval_prompt_and_model = prompt | eval_llm
        output = eval_prompt_and_model.invoke({"question": question, "answer": answer, "reference": reference})

        # Parse the output using the parser
        parsed_result = parser.invoke(output)

        # Store results
        eval_store_data.append(parsed_result)
        eval_store_data[index]["question"] = question
        eval_store_data[index]["reference"] = reference
        eval_store_data[index]["answer"] = answer  
    return eval_store_data

# Create a prompt 
def create_prompt(prompt_template, criteria):
    # Define output schema
    response_schemas = [
        ResponseSchema(name="evaluation", description="feedback on answer"),
        ResponseSchema(
            name=criteria,
            description="evaluation of answer, must be a percentage",
        ),
    ]

    # Define pydanthic output parser
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()
    
    eval_prompt = PromptTemplate(
        template = prompt_template.template,
        input_variables=["question", "answer", "reference"],
        partial_variables={"format_instructions": format_instructions},
    )
    return eval_prompt, output_parser

# def create_priority_prompt(prompt_template, criteria):
#     # Define output schema
#     response_schemas = [
#         ResponseSchema(name="evaluation", description="feedback on answer"),
#         ResponseSchema(
#             name=criteria,
#             description="evaluation of answer, must be a percentage",
#         ),
#     ]

#     # Define pydanthic output parser
#     output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
#     format_instructions = output_parser.get_format_instructions()
    
#     eval_prompt = PromptTemplate(
#         template = prompt_template.template,
#         input_variables=["answer_list", "reference_list"],
#         partial_variables={"format_instructions": format_instructions},
#     )
#     return eval_prompt, output_parser

def execute_benchmarks(prompt, criteria, headers):
    prompt, parser = create_prompt(prompt, criteria)
    if criteria == "prioritization": # Select only rows with prioritization flag
        testcases_df = testcases_df.loc[testcases_df['prioritization_flag'] == 1]
        
    store_data = evaluate_criteria(parser, testcases_df, eval_model, gemini_model, prompt)
    df = pd.DataFrame(store_data, columns=headers)
    return df

def calculate_delta(reference_data, criteria, criteria_header, dataframe):
    reference_data = reference_data.loc[:,["question", "reference", criteria_header]]
    output_df = pd.merge(dataframe, reference_data, how="inner", on=["question", "reference"])
    output_df[criteria] = output_df[criteria].str.rstrip('%').astype('float')/100.0
    output_df[criteria_header] = output_df[criteria_header].str.rstrip('%').astype('float')/100.0
    output_df['delta'] = output_df.apply(lambda x: abs(x[criteria] - x[criteria_header]), axis=1)
    return output_df
    
# Load test cases and data store
testcases_df = pd.read_csv("testcases_v1.csv")
manual_scores_df = pd.read_csv("reference_scoring_v1.csv")

# Define output headers
relevance_answer_headers = ["question", "reference", "answer", "evaluation", "relevance"]
depth_answer_headers = ["question", "reference", "answer", "evaluation", "depth"]
priority_answer_headers = ["question", "reference", "answer", "evaluation", "prioritization"]

# Pull latest prompt from LangSmith
relevance_prompt = hub.pull("benchmarking_relevance_v1")
depth_prompt = hub.pull("benchmarking_depth_v1")
extraction_prompt = hub.pull("benchmarking_keypoints_extraction_v1")
priority_prompt = hub.pull("benchmarking_prioritization_v1")

# Initialize models to test
gpt4_model  = ChatOpenAI(model="gpt-4o", temperature=0.5)
gemini_model = ChatGoogleGenerativeAI(model="gemini-pro")

# Initialize evaluator model
eval_model = ChatOpenAI(model="gpt-4o", temperature=0.0)

test_relevence = True
test_depth = True
test_priority = True

# Run relevance benchmark
if test_relevence:
    prompt, parser = create_prompt(relevance_prompt, "relevance")
    store_data = evaluate_criteria(parser, testcases_df, eval_model, gemini_model, prompt)
    rel_df = pd.DataFrame(store_data, columns=relevance_answer_headers)
    display(rel_df)
    %store rel_df
    
    # Calculate delta
    criteria_header = "nps_advisor_relevance"
    criteria = "relevance" 
    compare_df = calculate_delta(manual_scores_df, criteria, criteria_header, rel_df)

    # Output
    print("Relevance Delta: "+str(compare_df['delta'].mean()))
    compare_df.to_excel("rel_scoring.xlsx")

# Run depth benchmark
if test_depth:
    prompt, parser = create_prompt(depth_prompt, "depth")
    store_data = evaluate_criteria(parser, testcases_df, eval_model, gemini_model, prompt)
    dep_df = pd.DataFrame(store_data, columns=depth_answer_headers)
    display(dep_df)
    %store dep_df
    
    # Calculate delta
    criteria_header = "nps_advisor_depth"
    criteria = "depth" 
    compare_df = calculate_delta(manual_scores_df, criteria, criteria_header, dep_df)

    # Output
    print("Depth Delta: "+str(compare_df['delta'].mean()))
    compare_df.to_excel("dep_scoring.xlsx")
    
# Run priority benchmark
if test_priority:
    prompt, parser = create_prompt(priority_prompt, "prioritization")
    testcases_df = testcases_df.loc[testcases_df['prioritization_flag'] == 1] # Select only rows with prioritization flag
    store_data = evaluate_criteria(parser, testcases_df, eval_model, gemini_model, prompt)
    pri_df = pd.DataFrame(store_data, columns=priority_answer_headers)
    display(pri_df)
    %store pri_df

    # Calculate delta
    criteria_header = "nps_advisor_priority"
    criteria = "prioritization" 
    compare_df = calculate_delta(manual_scores_df, criteria, criteria_header, pri_df)

    # Output
    print("Prioritization Delta: "+str(compare_df['delta'].mean()))
    compare_df.to_excel("pri_scoring.xlsx")
    

,question,reference,answer,evaluation,relevance
0,Summarize the NPS for this year (2024) for Cir...,Average NPS of -17 in 2024. \n\nJanuary 2024: ...,"As of 2024, Circles.Life has experienced fluct...",The answer provides a detailed month-by-month ...,90%
1,What trends have Circles life observed in NPS ...,"""2017: +54 \n2018: +50\n2019: +45\n2020: +40\n...",### Circles.Life NPS Trends Over the Past 5 Ye...,The answer provides a detailed narrative of po...,0%
2,What are the top issues mentioned in the App S...,Network Reliability: A significant number of u...,Based on the reviews from the App Store and Go...,The answer covers several key issues mentioned...,57%
3,What emerging themes or trends for Circles.Lif...,"""Analyzing the provided NPS data and customer ...",Emerging themes and trends in customer feedbac...,The answer captures several key points from th...,66%
4,How many customers are passive in Circles.Life...,"Circles.Life has approximately 1,478 passive c...",The current number of passive customers for Ci...,The answer provides a detailed explanation of ...,20%
5,\nWhat are the top issues causing detractors f...,Network Coverage and Reliability: Customers ha...,Detractors of Circles.Life have expressed diss...,The answer addresses some of the issues mentio...,40%
6,Compare the NPS of Circles with other Singapor...,Circles.Life has achieved a Net Promoter Score...,"As of September 2024, Circles.Life Singapore h...",The answer provides a detailed comparison of C...,57%
7,What key drivers contribute to customers' loya...,4 Key Strategies:\n\nExceptional Customer Serv...,Customers' loyalty and advocacy for Circles.Li...,The answer covers several key drivers of custo...,45%
8,"Based on the reasons for NPS, which aspects sh...",Positive Aspects (Promoter Reasons):\nValue fo...,To improve the Net Promoter Score (NPS) for Ci...,The answer covers several key aspects mentione...,50%
9,What are the primary reasons for promoters' hi...,Value for Money: Affordable plans that provide...,Promoters of Circles.Life have provided severa...,The answer covers several key points related t...,33%


C:\Users\njeny\AppData\Roaming\Python\Python313\site-packages\IPython\extensions\storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/rel_df requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


Stored 'rel_df' (DataFrame)
Relevance Delta: 0.10966666666666666


,question,reference,answer,evaluation,depth
0,Summarize the NPS for this year (2024) for Cir...,Average NPS of -17 in 2024. \n\nJanuary 2024: ...,"As of 2024, Circles.Life has experienced fluct...",The answer provides a detailed month-by-month ...,70%
1,What trends have Circles life observed in NPS ...,"""2017: +54 \n2018: +50\n2019: +45\n2020: +40\n...",### Circles.Life NPS Trends Over the Past 5 Ye...,The answer provides a detailed narrative of po...,0%
2,What are the top issues mentioned in the App S...,Network Reliability: A significant number of u...,Based on the reviews from the App Store and Go...,The answer provides a comprehensive overview o...,66.67%
3,What emerging themes or trends for Circles.Lif...,"""Analyzing the provided NPS data and customer ...",Emerging themes and trends in customer feedbac...,The answer effectively covers most of the emer...,83%
4,How many customers are passive in Circles.Life...,"Circles.Life has approximately 1,478 passive c...",The current number of passive customers for Ci...,The answer provides a detailed breakdown of pa...,20%
5,\nWhat are the top issues causing detractors f...,Network Coverage and Reliability: Customers ha...,Detractors of Circles.Life have expressed diss...,The answer identifies three main issues: rewar...,40%
6,Compare the NPS of Circles with other Singapor...,Circles.Life has achieved a Net Promoter Score...,"As of September 2024, Circles.Life Singapore h...",The answer provides some insights into the fac...,43%
7,What key drivers contribute to customers' loya...,4 Key Strategies:\n\nExceptional Customer Serv...,Customers' loyalty and advocacy for Circles.Li...,The answer provides a comprehensive overview o...,70%
8,"Based on the reasons for NPS, which aspects sh...",Positive Aspects (Promoter Reasons):\nValue fo...,To improve the Net Promoter Score (NPS) for Ci...,The answer provides a comprehensive list of as...,70%
9,What are the primary reasons for promoters' hi...,Value for Money: Affordable plans that provide...,Promoters of Circles.Life have provided severa...,The answer provides several subpoints that ali...,33%


C:\Users\njeny\AppData\Roaming\Python\Python313\site-packages\IPython\extensions\storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/dep_df requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


Stored 'dep_df' (DataFrame)
Depth Delta: 0.16259999999999997


,question,reference,answer,evaluation,prioritization
0,Summarize the NPS for this year (2024) for Cir...,Average NPS of -17 in 2024. \n\nJanuary 2024: ...,"As of 2024, Circles.Life has experienced fluct...",The answer provides a detailed month-by-month ...,100%
1,What trends have Circles life observed in NPS ...,"""2017: +54 \n2018: +50\n2019: +45\n2020: +40\n...",### Circles.Life NPS Trends Over the Past 5 Ye...,The answer provides a detailed narrative of th...,40%
2,What are the top issues mentioned in the App S...,Network Reliability: A significant number of u...,Based on the reviews from the App Store and Go...,The answer provides a comprehensive list of is...,57%
3,What emerging themes or trends for Circles.Lif...,"""Analyzing the provided NPS data and customer ...",Emerging themes and trends in customer feedbac...,The answer identifies several key themes that ...,50%
4,How many customers are passive in Circles.Life...,"Circles.Life has approximately 1,478 passive c...",The current number of passive customers for Ci...,The answer provides a detailed analysis of the...,20%
5,\nWhat are the top issues causing detractors f...,Network Coverage and Reliability: Customers ha...,Detractors of Circles.Life have expressed diss...,The answer identifies three main issues: rewar...,40%


Stored 'pri_df' (DataFrame)
Prioritization Delta: 0.105


C:\Users\njeny\AppData\Roaming\Python\Python313\site-packages\IPython\extensions\storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/pri_df requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [22]:
import pandas as pd

output_df = pd.merge(rel_df, dep_df, how="inner", on=["question", "reference", "answer"])
output_df = pd.merge(output_df, pri_df, how="outer", on=["question", "reference", "answer"])     
display(output_df)
output_df["score"] = output_df.apply(lambda x: x["relevance"] * 0.5 + x["depth"] * 0.5 if not x["prioritization"] else x["relevance"] * 0.45 + x["depth"] * 0.45 + x["prioritization"] * 0.1)
output_df


,question,reference,answer,evaluation_x,relevance,evaluation_y,depth,evaluation,prioritization
0,\nWhat are the top issues causing detractors f...,Network Coverage and Reliability: Customers ha...,Detractors of Circles.Life have expressed diss...,The answer addresses some of the issues mentio...,40%,The answer identifies three main issues: rewar...,40%,The answer identifies three main issues: rewar...,40%
1,At which stages do customers encounter either ...,Onboarding:\nDelight: Promoters in the data of...,Customers encounter friction or delight at var...,The answer covers all the key points mentioned...,100%,The answer provides a comprehensive overview o...,83.33%,NaN,NaN
2,"Based on the reasons for NPS, which aspects sh...",Positive Aspects (Promoter Reasons):\nValue fo...,To improve the Net Promoter Score (NPS) for Ci...,The answer covers several key aspects mentione...,50%,The answer provides a comprehensive list of as...,70%,NaN,NaN
3,Compare the NPS of Circles with other Singapor...,Circles.Life has achieved a Net Promoter Score...,"As of September 2024, Circles.Life Singapore h...",The answer provides a detailed comparison of C...,57%,The answer provides some insights into the fac...,43%,NaN,NaN
4,Do certain customer segments experience signif...,Segment-Specific NPS Outcomes\nPlan Type:\n\nH...,"Yes, certain customer segments do experience s...",The answer identifies that certain customer se...,50%,The answer provides a comprehensive overview o...,50%,NaN,NaN
5,How do NPS scores vary across customer segment...,1. Geography (Roaming vs. Non-Roaming Segments...,NPS scores for Circles.Life vary significantly...,The answer provides a detailed analysis of NPS...,25%,The answer provides a detailed analysis of NPS...,50%,NaN,NaN
6,How many customers are passive in Circles.Life...,"Circles.Life has approximately 1,478 passive c...",The current number of passive customers for Ci...,The answer provides a detailed explanation of ...,20%,The answer provides a detailed breakdown of pa...,20%,The answer provides a detailed analysis of the...,20%
7,Is there a difference in NPS score between por...,The average NPS score for New Number customers...,"Yes, there is a difference in NPS scores betwe...",The answer provides a detailed analysis of the...,50%,The answer provides a detailed analysis of the...,50%,NaN,NaN
8,Summarize the NPS for this year (2024) for Cir...,Average NPS of -17 in 2024. \n\nJanuary 2024: ...,"As of 2024, Circles.Life has experienced fluct...",The answer provides a detailed month-by-month ...,90%,The answer provides a detailed month-by-month ...,70%,The answer provides a detailed month-by-month ...,100%
9,What are the primary reasons for promoters' hi...,Value for Money: Affordable plans that provide...,Promoters of Circles.Life have provided severa...,The answer covers several key points related t...,33%,The answer provides several subpoints that ali...,33%,NaN,NaN


KeyError: 'prioritization'